In [8]:
import bql
import pandas as pd
import numpy as np
bq = bql.Service()

In [9]:
#--------------------------#
#  BQL Editor - Filtragem  #
#--------------------------#

# (1) Coupon (Títulos de Credito)

# BQL String:
# get(ID()) 
# for(bonds('PETR4 BZ Equity'))

# BQL Object Model:
bql_item = bq.data.id()
bql_universe = bq.univ.bonds('PETR4 BZ Equity')
bql_request = bql.Request(bql_universe, bql_item)
bql_response = bq.execute(bql_request)
df = bql_response[0].df()
df.head()

,ORIG_IDS,ID()
ID,,
AO933589 Corp,PETR4 BZ Equity,AO933589 Corp
ZQ049596 Corp,PETR4 BZ Equity,ZQ049596 Corp
AX187074 Corp,PETR4 BZ Equity,AX187074 Corp
AX187049 Corp,PETR4 BZ Equity,AX187049 Corp
AX187079 Corp,PETR4 BZ Equity,AX187079 Corp


In [10]:
# (2) Filter example 1:
# Coupon + Yield filter:

# BQL String:
# get(id()) 
# for(filter(bonds('VALE3 BZ Equity'), cpn>5 and yield>3))

# BQL Object Model:
bql_item = bq.data.id()
bql_universe = bq.univ.filter(bq.univ.bonds('VALE3 BZ Equity'), bq.func.and_(bq.data.CPN() > 5, bq.data.YIELD() >= 3))
bql_request = bql.Request(bql_universe, bql_item)
bql_response = bq.execute(bql_request)
df = bql_response[0].df()
df.head()

,ORIG_IDS,ID()
ID,,
EF861855 Corp,VALE3 BZ Equity,EF861855 Corp
EJ350241 Corp,VALE3 BZ Equity,EJ350241 Corp
EI036018 Corp,VALE3 BZ Equity,EI036018 Corp
EC691515 Corp,VALE3 BZ Equity,EC691515 Corp
EK055420 Corp,VALE3 BZ Equity,EK055420 Corp


In [11]:
# Filter example 2:
# Members + PX_Last + BETWEEN

# BQL String:
# get(PX_LAST()) 
# for(filter(MEMBERS('IBOV Index'), BETWEEN(PX_LAST, lscalar=75, rscalar=125)))

# BQL Object Model:
bql_item = bq.data.px_last()
bql_universe = bq.univ.filter(bq.univ.members('IBOV Index'), bq.func.between(bq.data.px_last(), lscalar=75, rscalar=125))
bql_request = bql.Request(bql_universe, bql_item)
bql_response = bq.execute(bql_request)
df = bql_response[0].df()
df.head()

,DATE,CURRENCY,PX_LAST()
ID,,,
RADL3 BS Equity,2020-08-14,BRL,113.500000
MGLU3 BS Equity,2020-08-14,BRL,81.129997
BTOW3 BS Equity,2020-08-14,BRL,113.720001
BPAC11 BS Equity,2020-08-14,BRL,82.139999
CSAN3 BS Equity,2020-08-14,BRL,82.949997


In [12]:
# Filter example 3:
# Members + PX_Last + type uing in (categorical filter)

# BQL String:
# get(PX_LAST()) 
# for(filter(MEMBERS('IBOV Index'), in(eqy_fund_ind, ['Bank', 'Financial', 'Insurance'] )))
bql_item = bq.data.px_last()

my_dict = {'Bank': [],
           'Financial': [],
           'Insurance': []}

for k in my_dict.keys():
    bql_universe = (bq.univ.filter(bq.univ.members('IBOV Index'), bq.data.eqy_fund_ind() == k))
    bql_request = bql.Request(bql_universe, bql_item)
    bql_response = bq.execute(bql_request)
    df = bql_response[0].df()
    my_dict[k] = df

df = pd.concat(my_dict.values())
df

,DATE,CURRENCY,PX_LAST()
ID,,,
BBAS3 BS Equity,2020-08-14,BRL,33.080002
ITUB4 BS Equity,2020-08-14,BRL,24.700001
SANB11 BS Equity,2020-08-14,BRL,29.090000
BBDC3 BS Equity,2020-08-14,BRL,19.559999
BBDC4 BS Equity,2020-08-14,BRL,21.139999
BBSE3 BS Equity,2020-08-14,BRL,27.080000
BPAC11 BS Equity,2020-08-14,BRL,82.139999
SULA11 BS Equity,2020-08-14,BRL,47.599998
IRBR3 BS Equity,2020-08-14,BRL,7.760000


In [13]:
# Filter example 4: Price threshold over certein period (Time Series)
# Members + PX_Last + Raw Prices:

# BQL String:
# get(ID()) 
# for(filter(MEMBERS('IBOV Index'), all(PX_LAST(dates=range(start=2018-01-01, end=2019-12-31), ca_adj=RAW)>30)))

# BQL Object Model:
bql_item = bq.data.id()
bql_universe = bq.univ.filter(bq.univ.members('IBOV Index'), bq.func.all(bq.data.px_last(dates=bq.func.range(start='2018-01-01', end='2019-12-31'), ca_adj='RAW') > 30))
bql_request = bql.Request(bql_universe, bql_item)
bql_response = bq.execute(bql_request)
df = bql_response[0].df()
df

,Weights,Positions,ORIG_IDS,ID()
ID,,,,
RADL3 BS Equity,1.282766,20.742543,IBOV INDEX,RADL3 BS Equity
ITUB4 BS Equity,6.374665,473.856268,IBOV INDEX,ITUB4 BS Equity
VALE3 BS Equity,10.941597,329.201081,IBOV INDEX,VALE3 BS Equity
EGIE3 BS Equity,0.603811,25.481340,IBOV INDEX,EGIE3 BS Equity
MGLU3 BS Equity,2.925635,66.183408,IBOV INDEX,MGLU3 BS Equity
VIVT4 BS Equity,1.133218,41.513187,IBOV INDEX,VIVT4 BS Equity
PCAR3 BS Equity,0.560420,15.657573,IBOV INDEX,PCAR3 BS Equity
IRBR3 BS Equity,0.528855,125.079277,IBOV INDEX,IRBR3 BS Equity
CSAN3 BS Equity,0.693397,15.341783,IBOV INDEX,CSAN3 BS Equity


In [14]:
# Filter example 5: EquitiesUniv 
# Equities Univ + country + Sector
# GICS_SECTOR_NAME: Global Industry Classification Standard

# BQL String:
# get(ID()) 
# for(filter(EQUITIESUNIV(['active', 'primary']), GICS_SECTOR_NAME == 'Information Technology' and CNTRY_OF_DOMICILE == 'BR' ))

# BQL Object Model:
bql_item = bq.data.id()
bql_universe = bq.univ.filter(bq.univ.equitiesuniv(['active', 'primary']), bq.func.and_(bq.data.gics_sector_name() == 'Information Technology', bq.data.cntry_of_domicile() == 'BR'))
bql_request = bql.Request(bql_universe, bql_item)
bql_response = bq.execute(bql_request)
df = bql_response[0].df()
df

,ID()
ID,
CIEL3 BZ Equity,CIEL3 BZ Equity
TOTS3 BZ Equity,TOTS3 BZ Equity
CARD3 BZ Equity,CARD3 BZ Equity
POSI3 BZ Equity,POSI3 BZ Equity
SQIA3 BZ Equity,SQIA3 BZ Equity
LINX3 BZ Equity,LINX3 BZ Equity
PAGS US Equity,PAGS US Equity
STNE US Equity,STNE US Equity
LWSA3 BZ Equity,LWSA3 BZ Equity
